# Datathon data-extraction

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data'
ZIP_NAME = 'smart_destinations_v2.zip'

## Unzip

In [2]:
import zipfile
from pathlib import Path

BASE_DIR = Path(BASE_DIR)

with zipfile.ZipFile(BASE_DIR.joinpath(ZIP_NAME), 'r') as zip_ref:
    zip_ref.extractall(BASE_DIR)

In [3]:
list(BASE_DIR.glob('*'))

[PosixPath('/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/datos'),
 PosixPath('/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/.DS_Store'),
 PosixPath('/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/ocean_elements.zip'),
 PosixPath('/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/ocean_fondos.zip'),
 PosixPath('/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/smart_destinations_v2.zip')]

In [4]:
import pandas as pd

aux = []
for x in BASE_DIR.glob('**/*.csv'):
    file_path = str(x)
    file_folder = file_path.split('/')[-2]
    file_name = file_path.split('/')[-1]
    sub_df = pd.read_csv(file_path, iterator=True)
    one_row = sub_df.get_chunk(1)
    cols = one_row.columns.tolist()
    aux.append((file_folder, file_name, len(cols), cols))

print(aux[:3])

[('CM28', 'maestro_cm28.csv', 22, ['Anyo', 'Isla', 'Municipio', 'Modalidad', 'Clasificacion', 'Plazas', 'Plazas_por_isla', 'Perc_Plazas', 'Establecimientos', 'Alojativos_por_isla', 'Perc_Establecimientos', 'Plazas_prev', 'Plazas_por_isla_prev', 'Perc_Plazas_prev', 'Establecimientos_prev', 'Alojativos_por_isla_prev', 'Perc_Establecimientos_prev', 'max_anyo', 'total_establecimientos', 'total_plazas', 'total_establecimientos_prev', 'total_plazas_prev']), ('CM07_1', 'recursos.csv', 6, ['nombre', 'tlfn', 'latitud', 'longitud', 'tipo', 'max_fecha']), ('CM19', 'maestro_cm19.csv', 9, ['lugar', 'origen', 'anyo', 'mes', 'valor', 'valorPrev', 'indicador', 'dimension', 'ultima_fecha'])]


In [5]:
df = pd.DataFrame(aux, columns=['folder','file_name','n_columns','columns'])
df['clean_columns'] = df['columns'].map(lambda x: ' '.join(x).replace('_',' '))
df[['columns','clean_columns']].sample(6)

,columns,clean_columns
248,"[indicador, estacion, nombre, provincia, latit...",indicador estacion nombre provincia latitud lo...
327,"[pais, municipio, periodo, valor]",pais municipio periodo valor
106,"[zipcode, country, day, cards]",zipcode country day cards
319,"[indicador, categoria, origen, municipio, peri...",indicador categoria origen municipio periodo v...
249,"[indicador, estacion, nombre, provincia, latit...",indicador estacion nombre provincia latitud lo...
295,"[geo, time, value, indicador, region]",geo time value indicador region


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

X = cv.fit_transform(df['clean_columns']).toarray()
X = pd.DataFrame(X, columns=cv.get_feature_names(), index=df['file_name'])
X.sample(6)

,actividad,actual,administrativa,advanced,age,airport,alojamiento,alojativos,amount,anual,...,ultima,valor,valorprev,value,venue,visitantes,visitor,visitors,week,zipcode
file_name,,,,,,,,,,,,,,,,,,,,,
country_visitors.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0
cards_35009.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
cards_35212.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
avg_amount_35340.csv,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
avg_amount_35240.csv,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
tasa_ocupacion_extrahoteleras%20.csv,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [7]:
from sklearn.cluster import KMeans

km = KMeans(13)
X['cluster'] = km.fit_predict(X)
df = df.set_index('file_name').join(X[['cluster']]).sort_values('cluster')
df.sample()

,folder,n_columns,columns,clean_columns,cluster
file_name,,,,,
cards_35333.csv,CM01_3,4,"[zipcode, country, day, cards]",zipcode country day cards,2


In [8]:
df.to_csv(BASE_DIR.parent.joinpath('all_files_columns.csv'), sep='\t', encoding='utf-16')